In [1]:
import sys
sys.path.append('../src')

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

import pandas as pd

from functions import monte_carlo_sim

from data_cleaning import DataCleaning

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
df = pd.read_csv('../data/turf.csv')

In [3]:
df.head()

,Course,Course Direction,Distance (y),Race Type,Class,Going,Race Date,Race Time,Racecard No.,Draw,Draw IV,Position,Horse,DTN,DTW,Jockey,Jockey Claim,Trainer,Age,Weight (pounds),Penalty,Weight Rank,DSLR,Equip,Sex Abbrev,Favourite,BF Decimal SP,"Won (1=Won, 0=Lost)","Place (1=Placed, 0=UnPlaced)",PRC Average,PRC Last Run,PRC 2nd Last Run,PRC 3rd Last Run,Pace Abbrev,Pace Rating,Pace Rating Rank,OR,OR Rank,Tissue Odds Text,Tissue Odds Decimal,Tissue Rating,Tissue Rating Rank,TIS+PFR,PFR,PFR Rank,Main,Main Rank,Main Stats,Main+Stats Rank,Stats,Stats Rank,Trn Stats,Trainer Stats Rank,Jky Stats,Jockey Stats Rank,TrnJky Stats,Trainer/Jky Stats Rank,Hrs Stats,Horse Stats Rank,UR1,UR1 Rank,UR2,UR2 Rank,UR3,UR3 Rank,UR4,UR4 Rank,HA Career Speed Rating,HA Career Speed Rating Rank,HA Last 1 Year Speed Rating,HA Last 1 Year Speed Rating Rank,MR Career Speed Rating,MR Career Speed Rating Rank,MR Last 1 Year Speed Rating,MR Last 1 Year Speed Rating Rank,MR Last 3 Runs Speed Rating,MR Last 3 Runs Speed Rating Rank,LTO Speed Rating,LTO Speed Rating Rank,2nd LTO Speed Rating,2nd LTO Speed Rating Rank,3rd LTO Speed Rating,3rd LTO Speed Rating Rank,4th LTOt Speed Rating,4th LTO Speed Rating Rank,Horse Time In Seconds,dyHA,dyLTO,dyAVG,dyAVG(x),dyMR,dyMR(x),dy2HA,Runs Before,Won Before,Distance In Yards LTO (Same Code),Official Rating LTO (Same Code),FLAT TURF Runs Before,NHF Wins Before,Distance To Winner LTO,User Rating 1 LTO,User Rating 2 LTO,User Rating 3 LTO,User Rating 4 LTO,Speed Rating LTO,PRC Average LTO,PRC Last Run LTO,PRC 2nd Last Run LTO,PRC 3rd Last Run LTO,Today's Going Wins,Today's Going Places,Today's Going Runs,Today's Going PRB,Today's Going PRB^2,Today's Going 50% SR,Today's Distance Wins,Today's Distance Places,Today's Distance Runs,Today's Distance PRB,Today's Distance PRB^2,Today's Distance 50% SR,Today's Class Wins,Today's Class Places,Today's Class Runs,Today's Class PRB,Today's Class PRB^2,Today's Class 50% SR,Today's Course Wins,Today's Course Places,Today's Course Runs,Today's Course PRB,Today's Course PRB^2,Today's Course 50% SR,Pace Abbreviation LTO (Same Code)
0,Ascot,RH,1100,Turf,A,Good,05/10/2024,13:50,1,5,1.05,6.0,Democracy Dilemma,7.0,16.049999,Jamie Spencer,0,Robert Cowell,4,132,0,1,14.0,NaN,g,NaN,6.79,0,0,94,90,97,94,L,12,1,106,1,9/2,5.5,36,3,80,44,3,334,3,663,3,329,4,83,4,86,5,82,6,78,3,215,3,NaN,1,NaN,1,NaN,1,81,2,81,2,65,5,75,1,79,1,81,2,79,3,75,1,67,2,64.669998,81.0,81.0,62.0,78.0,64.0,79.0,79.0,29,7,1134,106.0,20,0,1.65,147.0,NaN,NaN,NaN,81.0,94.0,97.0,94.0,91.0,1,0,3,57.41,43.31,66.67,7,5,24,67.26,53.97,70.83,1,0,6,48.46,32.13,50.00,0,0,1,21.74,4.73,0.00,L
1,Ascot,RH,1100,Turf,A,Good,05/10/2024,13:50,2,3,0.67,5.0,Go Athletico,6.0,9.050000,D Egan,0,Adrian McGuinness,6,129,0,2,6.0,NaN,g,NaN,10.38,0,0,88,84,86,94,P,4,3,104,2,9/2,5.5,35,4,64,29,5,317,4,635,5,318,5,78,6,76,6,88,3,76,4,103,5,NaN,1,NaN,1,NaN,1,78,5,71,6,72,2,70,4,70,4,65,6,70,5,71,4,0,4,62.919998,78.0,65.0,72.0,68.0,72.0,70.0,75.0,7,2,1320,104.0,7,0,3.35,93.0,NaN,NaN,NaN,65.0,93.0,86.0,94.0,98.0,1,1,2,92.86,86.73,50.00,1,2,4,73.23,60.65,25.00,2,3,7,71.84,61.09,28.57,0,0,0,NaN,NaN,NaN,P
2,Ascot,RH,1100,Turf,A,Good,05/10/2024,13:50,3,1,1.24,1.0,Rumstar,0.0,0.000000,Hector Crouch,0,Jonathan Portman,4,129,0,2,21.0,NaN,g,NaN,7.08,1,0,87,93,76,91,P,4,3,101,5,14/1,15.0,13,6,32,19,6,281,6,579,6,298,6,80,5,91,2,83,5,44,6,93,6,NaN,1,NaN,1,NaN,1,78,5,78,5,69,4,70,4,70,4,70,4,66,6,72,3,64,3,60.660000,78.0,70.0,61.0,69.0,66.0,70.0,76.0,19,3,1243,101.0,18,0,0.70,29.0,NaN,NaN,NaN,70.0,78.0,76.0,91.0,68.0,0,0,0,NaN,NaN,NaN,2,1,6,65.69,56.17,66.67,1,0,9,50.49,31.89,44.44,0,0,3,34.72,18.58,33.33,P
3,Ascot,RH,1100,Turf,A,Good,05/10/2024,13:50,4,4,1.43,2.0,Beautiful Diamond,2.0,2.000000,Clifford Lee,0,K R Burke,3,127,0,4,20.0,NaN,f,3.0,4.05,0,1,95,91,96,97,P,6,2,104,2,7/2,4.5,44,2,90,46,2,342,2,683,2,341,3,91,3,87,4,96,2,67,5,231,2,NaN,1,NaN,1,NaN,1,80,4,80,4,74,1,75,1,75,2,74,3,80,2,75,1,75,1,61.160000,80.0,74.0,72.0,76.0,74.0,75.0,78.0,9,3,1100,104.0,9,0,4.20,31.0

In [4]:
df['race_id'] = df.groupby(['Race Date', 'Race Time']).ngroup()

## train a random forest classifier to obtain probalities 

In [5]:
# Select only numeric columns
# numeric_cols = df.select_dtypes(include=['number']).columns

numeric_cols = ["PFR","HA Career Speed Rating", "LTO Speed Rating", "Today's Going PRB"]

# Normalize only numeric columns
normalized_df = DataCleaning.normalize_columns(df, numeric_cols)

In [6]:
# Replace NaN in all numeric columns
cleaned_df = DataCleaning.replace_nan(normalized_df, numeric_cols)

cleaned_df.head()

,Course,Course Direction,Distance (y),Race Type,Class,Going,Race Date,Race Time,Racecard No.,Draw,Draw IV,Position,Horse,DTN,DTW,Jockey,Jockey Claim,Trainer,Age,Weight (pounds),Penalty,Weight Rank,DSLR,Equip,Sex Abbrev,Favourite,BF Decimal SP,"Won (1=Won, 0=Lost)","Place (1=Placed, 0=UnPlaced)",PRC Average,PRC Last Run,PRC 2nd Last Run,PRC 3rd Last Run,Pace Abbrev,Pace Rating,Pace Rating Rank,OR,OR Rank,Tissue Odds Text,Tissue Odds Decimal,Tissue Rating,Tissue Rating Rank,TIS+PFR,PFR,PFR Rank,Main,Main Rank,Main Stats,Main+Stats Rank,Stats,Stats Rank,Trn Stats,Trainer Stats Rank,Jky Stats,Jockey Stats Rank,TrnJky Stats,Trainer/Jky Stats Rank,Hrs Stats,Horse Stats Rank,UR1,UR1 Rank,UR2,UR2 Rank,UR3,UR3 Rank,UR4,UR4 Rank,HA Career Speed Rating,HA Career Speed Rating Rank,HA Last 1 Year Speed Rating,HA Last 1 Year Speed Rating Rank,MR Career Speed Rating,MR Career Speed Rating Rank,MR Last 1 Year Speed Rating,MR Last 1 Year Speed Rating Rank,MR Last 3 Runs Speed Rating,MR Last 3 Runs Speed Rating Rank,LTO Speed Rating,LTO Speed Rating Rank,2nd LTO Speed Rating,2nd LTO Speed Rating Rank,3rd LTO Speed Rating,3rd LTO Speed Rating Rank,4th LTOt Speed Rating,4th LTO Speed Rating Rank,Horse Time In Seconds,dyHA,dyLTO,dyAVG,dyAVG(x),dyMR,dyMR(x),dy2HA,Runs Before,Won Before,Distance In Yards LTO (Same Code),Official Rating LTO (Same Code),FLAT TURF Runs Before,NHF Wins Before,Distance To Winner LTO,User Rating 1 LTO,User Rating 2 LTO,User Rating 3 LTO,User Rating 4 LTO,Speed Rating LTO,PRC Average LTO,PRC Last Run LTO,PRC 2nd Last Run LTO,PRC 3rd Last Run LTO,Today's Going Wins,Today's Going Places,Today's Going Runs,Today's Going PRB,Today's Going PRB^2,Today's Going 50% SR,Today's Distance Wins,Today's Distance Places,Today's Distance Runs,Today's Distance PRB,Today's Distance PRB^2,Today's Distance 50% SR,Today's Class Wins,Today's Class Places,Today's Class Runs,Today's Class PRB,Today's Class PRB^2,Today's Class 50% SR,Today's Course Wins,Today's Course Places,Today's Course Runs,Today's Course PRB,Today's Course PRB^2,Today's Course 50% SR,Pace Abbreviation LTO (Same Code),race_id
0,Ascot,RH,1100,Turf,A,Good,05/10/2024,13:50,1,5,1.05,6.0,Democracy Dilemma,7.0,16.049999,Jamie Spencer,0,Robert Cowell,4,132,0,1,14.0,NaN,g,NaN,6.79,0,0,94,90,97,94,L,12,1,106,1,9/2,5.5,36,3,80,0.416667,3,334,3,663,3,329,4,83,4,86,5,82,6,78,3,215,3,NaN,1,NaN,1,NaN,1,0.910112,2,81,2,65,5,75,1,79,1,0.910112,2,79,3,75,1,67,2,64.669998,81.0,81.0,62.0,78.0,64.0,79.0,79.0,29,7,1134,106.0,20,0,1.65,147.0,NaN,NaN,NaN,81.0,94.0,97.0,94.0,91.0,1,0,3,0.5741,43.31,66.67,7,5,24,67.26,53.97,70.83,1,0,6,48.46,32.13,50.00,0,0,1,21.74,4.73,0.00,L,1157
1,Ascot,RH,1100,Turf,A,Good,05/10/2024,13:50,2,3,0.67,5.0,Go Athletico,6.0,9.050000,D Egan,0,Adrian McGuinness,6,129,0,2,6.0,NaN,g,NaN,10.38,0,0,88,84,86,94,P,4,3,104,2,9/2,5.5,35,4,64,0.291667,5,317,4,635,5,318,5,78,6,76,6,88,3,76,4,103,5,NaN,1,NaN,1,NaN,1,0.876404,5,71,6,72,2,70,4,70,4,0.730337,6,70,5,71,4,0,4,62.919998,78.0,65.0,72.0,68.0,72.0,70.0,75.0,7,2,1320,104.0,7,0,3.35,93.0,NaN,NaN,NaN,65.0,93.0,86.0,94.0,98.0,1,1,2,0.9286,86.73,50.00,1,2,4,73.23,60.65,25.00,2,3,7,71.84,61.09,28.57,0,0,0,NaN,NaN,NaN,P,1157
2,Ascot,RH,1100,Turf,A,Good,05/10/2024,13:50,3,1,1.24,1.0,Rumstar,0.0,0.000000,Hector Crouch,0,Jonathan Portman,4,129,0,2,21.0,NaN,g,NaN,7.08,1,0,87,93,76,91,P,4,3,101,5,14/1,15.0,13,6,32,0.208333,6,281,6,579,6,298,6,80,5,91,2,83,5,44,6,93,6,NaN,1,NaN,1,NaN,1,0.876404,5,78,5,69,4,70,4,70,4,0.786517,4,66,6,72,3,64,3,60.660000,78.0,70.0,61.0,69.0,66.0,70.0,76.0,19,3,1243,101.0,18,0,0.70,29.0,NaN,NaN,NaN,70.0,78.0,76.0,91.0,68.0,0,0,0,0.0000,NaN,NaN,2,1,6,65.69,56.17,66.67,1,0,9,50.49,31.89,44.44,0,0,3,34.72,18.58,33.33,P,1157
3,Ascot,RH,1100,Turf,A,Good,05/10/2024,13:50,4,4,1.43,2.0,Beautiful Diamond,2.0,2.000000,Clifford Lee,0,K R Burke,3,127,0,4,20.0,NaN,f,3.0,4.05,0,1,95,91,96,97,P,6,2,104,2,7/2,4.5,44,2,90,0.433333,2,342,2,683,2,341,3,91,3,87,4,96,2,67,5,231,2,NaN,1,NaN,1,NaN,1,0.898876,4,80,4,74,1,75,1,75,2

In [7]:
train_data, test_data = DataCleaning.split_data(df = cleaned_df)

X_train = train_data[numeric_cols]
y_train = train_data['Won (1=Won, 0=Lost)']

X_test = test_data[numeric_cols]
y_test = test_data['Won (1=Won, 0=Lost)']

race_id_train = train_data["race_id"].copy()
race_id_test = test_data["race_id"].copy()

KeyError: 'race_id'

In [ ]:
X_train.head()

,PFR,HA Career Speed Rating,LTO Speed Rating,Today's Going PRB
0,0.416667,0.910112,0.910112,0.5741
1,0.291667,0.876404,0.730337,0.9286
2,0.208333,0.876404,0.786517,0.0000
3,0.433333,0.898876,0.831461,0.5000
4,0.375000,0.910112,0.752809,0.6667


In [ ]:
# Initialize and fit the RandomForest model
model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

df_test = X_test.copy()
df_test['Actual'] = y_test
df_test['Predicted_Prob'] = model.predict_proba(X_test)[:, 1]


In [ ]:
df_test.head(5)

,PFR,HA Career Speed Rating,LTO Speed Rating,Today's Going PRB,Actual,Predicted_Prob
316,0.391667,0.910112,0.842697,0.7733,0,0.120
317,0.383333,0.876404,0.820225,0.6667,0,0.155
318,0.250000,0.842697,0.764045,0.7292,1,0.095
319,0.325000,0.898876,0.741573,0.6941,0,0.070
320,0.350000,0.887640,0.764045,0.5598,0,0.095


## compute the monte carlo simulation

In [ ]:
results = []
for race_id, race_df in df_test.groupby("race_id"):
    res = monte_carlo_sim(race_df, n_sims=5000)
    res["race_id"] = race_id
    results.append(res)

final_results = pd.concat(results)

KeyError: 'race_id'